In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import spacy
from spacy.util import minibatch, compounding
import random

nlp = spacy.load('el_core_news_md')
df = pd.read_csv('../data/greek_fake_news.csv')
df.replace(to_replace='[\n\r\t]', value=' ', regex=True, inplace=True)

In [2]:
def load_data(train_data, limit=0, split=0.8):
    random.shuffle(train_data)
    train_data = train_data[-limit:]
    texts, labels = zip(*train_data)
    cats = [{"REAL": not bool(y), "FAKE": bool(y)} for y in labels]
    split = int(len(train_data) * split)
    
    return (texts[:split], cats[:split]), (texts[split:], cats[split:])

def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if label == "FAKE":
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.0
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.0
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score}


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    100 non-null    object
 1   text     100 non-null    object
 2   source   100 non-null    object
 3   url      100 non-null    object
 4   is_fake  100 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 4.0+ KB


In [4]:
textcat=nlp.create_pipe( "textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"})
nlp.add_pipe(textcat, last=True)
nlp.pipe_names


['tagger', 'parser', 'ner', 'textcat']

In [5]:
textcat.add_label("REAL")
textcat.add_label("FAKE")

1

In [6]:
df['tuples'] = df.apply(lambda row: (row['text'], row['is_fake']), axis=1)
train = df['tuples'].tolist()

In [7]:
(train_texts, train_cats), (dev_texts, dev_cats) = load_data(train, split=0.9)

train_data = list(zip(train_texts,[{'cats': cats} for cats in train_cats]))


In [8]:
n_iter = 20
# Disabling other components
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()

    print("Training the model...")
    print('{:^5}\t{:^5}\t{:^5}\t{:^5}'.format('LOSS', 'P', 'R', 'F'))

    # Performing training
    for i in range(n_iter):
        losses = {}
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)

      # Calling the evaluate() function and printing the scores
        with textcat.model.use_params(optimizer.averages):
            scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
        print('{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}'  
              .format(losses['textcat'], scores['textcat_p'],
                      scores['textcat_r'], scores['textcat_f']))

Training the model...
LOSS 	  P  	  R  	  F  
0.669	0.714	1.000	0.833
0.346	0.714	1.000	0.833
0.382	0.833	1.000	0.909
0.378	0.714	1.000	0.833
0.120	0.833	1.000	0.909
0.068	0.833	1.000	0.909
0.025	0.714	1.000	0.833
0.004	0.714	1.000	0.833
0.001	0.833	1.000	0.909
0.004	0.714	1.000	0.833
0.023	0.714	1.000	0.833
0.005	0.714	1.000	0.833
0.001	0.714	1.000	0.833
0.003	0.714	1.000	0.833
0.003	0.714	1.000	0.833
0.016	0.714	1.000	0.833
0.004	0.714	1.000	0.833
0.024	0.714	1.000	0.833
0.005	0.714	1.000	0.833
0.000	0.833	1.000	0.909


In [9]:
test = '''
Η Κυβερνητική ανακοίνωση να μειώσει τα Τακτικά Χειρουργεία εως 80% ΣΚΟΤΩΝΕΙ περισσότερους Ελληνες Πολίτες…ειναι η Κυβέρνηση που νοιάζεται για σένα. Χιλιάδες συμπολίτες μας περιμένουν να νοσηλευτούν να θεραπευτούν και να χειρουργηθούν απο τις γνωστές σοβαρές ασθένειες …και δεν μπορουν να περιμένουν.
Οσοι δεν εχουν την Οικονομική δυνατότητα να πάνε στον Ιδιωτικό τομεα το Κράτος του ΜΗΤΣΟΤΑΚΗ-ΧΑΡΔΑΛΙΑ δεν νοιάζεται..Σου λέει ΠΕΘΑΝΕ !! Άνοιξε εναν Λάκκο και πέσε μέσα!!
Γιατι? Γιατί τώρα οι ΘΕΑΤΡΙΝΟΙ της Κυβέρνησης παίζουν το ΕΡΓΟ του Κορωνοιού ..Του Μπαμπούλα της Νέας Τάξης ..Τώρα και με νέες στολές Διαστημικές για να θυμίζουν Χολιγουντιανές Ταινίες επιστημονικής Φαντασίας.
ΜΕ ΤΟ ΜΈΤΡΟ ΑΥΤΟ ΓΙΝΕΤΑΙ Η ΕΞΟΝΤΩΣΗ ΤΟΥ ΕΛΛΗΝΑ ΠΟΛΙΤΗ ..ΕΧΟΥΝ ΠΟΛΛΟΥΣ ΤΡΟΠΟΥΣ ΟΙ ΣΩΤΗΡΕΣ ΤΗΣ ΝΔ..
'''

doc = nlp(test)
doc.cats

{'REAL': 1.9296246378530668e-08, 'FAKE': 1.0}

In [10]:
with nlp.use_params(optimizer.averages):
            nlp.to_disk('../model')
        